In [48]:
import pandas as pd
import re
import datetime
import os
import subprocess

#Formatting .describe() calls s.t. floats are displayed in non-scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [26]:
file_name_l = ['/home/aleksey/Downloads/CSV_DATA/Jul_Dec_2019/sends_july_dec_2019_2000',
               '/home/aleksey/Downloads/CSV_DATA/Jan_Jun_2020/sends_jan_june_2020_2000',
               '/home/aleksey/Downloads/CSV_DATA/Jul_Sep_2020/sends_july_sept_2020_2000']

col_names = ['riid','send_dt','launch_id','sub_sourcedev','opened','unsub',
             'rev_3dv2','aq_dt','aq_mo','aq_dow','aq_period','sends_since_last_open',
             'wk1_opens','wk4_opens','mo3_opens','mo6_opens','yr1_opens',
             'wk1_clicks','wk4_clicks','mo3_clicks','mo6_clicks',
             'rev_dec','rev_raw','snds_dec','snds_opens_dec','wk1_vis',
             'wk4_vis','mo3_vis','mo6_vis','wk1_pv','wk4_pv','mo3_pv','mo6_pv',
             'wk1_secs','wk4_secs','mo3_secs','mo6_secs','prev_optouts',
             'last_order','last_click','last_cart','last_visit','last_ded_open',
             'sent_time','offer_signature_id','dynamic_content_signature_id',
             'message_size','open_cnt','first_open_time','last_open_time',
             'hrs_to_first_open','hrs_to_last_open','click_cnt','first_click_time',
             'last_click_time','offer_category','hrs_to_first_click','hrs_to_last_click',
             'opt_out_cnt','first_opt_out_time','last_opt_out_time','opt_out_source',
             'opt_out_reason','hrs_to_first_opt_out','hrs_to_last_opt_out','campaign_id',
             'subject','marketing_strategy','campaign_type']
categories = ['Brand', 'Core', 'Dedicated', 'InnovationSpotlight', 'NewArrivals', 'ProductSpotlight', 'Replen', 'Tops', 'Trend']

In [27]:
def rew(row):
    sl = -1 * (1+max(row['sends_since_last_open'], 0))
    #op = 5  * max(row['opened'], 0)
    if row['rev_3dv2'] > 0:
        cl = 10
    else:
        cl = 0
    un = -50 * max(row['unsub'], 0)
    fu = -50 * row['frequency_score'] * max(row['unsub'],0)
    return sl+cl+un+fu

In [28]:
def optimal(row):
    if row['opened'] == 1 and row['unsub'] == 0:
        optimal = 1
    else:
        optimal = 0
    return optimal

In [29]:
def campaign_t(row):
    for category in categories:
        if category in row['campaign_type']:
            return category
    return 'Other'

In [40]:
def dt(row):
#    print (row['aq_dt'])
    d = datetime.timedelta(days = row['aq_dt'])
    return d

In [30]:
#Too many questions around what to keep, what to drop...
#Hence, currently default behaviour of drop
#Using a control variable dictionary to modify behaviour for relevant column groups
control_variable = {'marketing': True, 'subject':True, 'campaign_type':False,
                    'aq': True,'site': True,'last':True, 'promo_ids': True,
                    'email_cnts':True, 'email_times':True, 'opt_out':True, 
                    'offer_cat':True}

In [47]:
# Split into files with about 10**6 lines each: not chronologically sorted
chunksize = 10 ** 6
# Sparse columns, agreed upon to drop
col_drop_names = ["send_dt", "sub_sourcedev", "rev_dec", "rev_raw", "snds_dec", "snds_opens_dec", "dynamic_content_signature_id"]
# Base columns for calculating recency/frequency/retention scores
col_drop_scores = ["wk1_opens", "wk4_opens", "mo3_opens","mo6_opens","yr1_opens", "wk1_clicks", "wk4_clicks", "mo3_clicks", "mo6_clicks"]

ordered_cols = ['riid', 'retention_score', 'frequency_score', 'days_subscr', 'aq_year', 'aq_week', 'aq_dow', 'aq_period', 'campaign_id', 'campaign_category', 'discount', 'promo', 'sale', 'message_size', 'sent_week', 'sent_dayofweek', 'sent_hr', 'opened','unsub', 'rev_3dv2', 'reward', 'optimal_action']
ordered_rows = ['sent_week', 'sent_dayofweek', 'sent_hr', 'campaign_id']
days_in_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31] # 2020 was a leap year

for filename in file_name_l:
    print (filename)
    ind = 0
    reader = pd.read_csv(filename, sep="\t", header=None, names=col_names, iterator=True, chunksize=chunksize)
    for df_sample in reader:
        print (ind)
        ind += 1
        
        df_sample.drop(columns=col_drop_names, inplace=True)

        #Retention Score
        df_sample['retention_score'] = df_sample.sends_since_last_open.apply(lambda x: 28/max(1,x)) 

        #Frequency Score
        df_sample["frequency_score"] = df_sample['wk1_opens'] + df_sample['wk4_opens'] + df_sample['mo3_opens'] + df_sample['mo6_opens']

        df_sample.drop(columns=col_drop_scores, inplace=True)

        # Get date and time sent features
        datetimes = pd.to_datetime(df_sample.sent_time)
        df_sample.drop(columns=["sent_time"], inplace=True)
        df_sample["sent_hr"] = datetimes.dt.hour
        df_sample["sent_dayofweek"] = datetimes.dt.dayofweek
        df_sample["sent_week"] = datetimes.dt.dayofyear // 7
        
        # Get duration subscribed (in days), as well as acquisition year and week
        df_sample["days_subscr"] = df_sample["aq_dt"]
        df_sample["dt"] = df_sample.apply(lambda row: dt(row), axis=1)
        df_sample["aq_year"] = (datetimes - df_sample["dt"]).dt.year
        df_sample["aq_week"] = (datetimes - df_sample["dt"]).dt.dayofyear // 7
        df_sample.drop(columns=['aq_dt', 'aq_mo', 'dt'], inplace=True)
        del datetimes

        
        if control_variable["marketing"]:
            #Drop the column
            df_sample.drop(columns=["marketing_strategy"], inplace=True)

        if control_variable["campaign_type"]:
            #Drop the campaign_type column 
            df_sample.drop(columns=["campaign_type"], inplace=True)
        else:
        #Do stops and then drop the column
        #Eliminate non-US rows
            US_mask = df_sample.campaign_type.str.contains('_US_')
            df_sample = df_sample[US_mask]

            #Get campaign category column
            df_sample["campaign_category"] = df_sample.apply(lambda row: campaign_t(row), axis=1)
  
            #Create discount columns
            df_sample["discount"] = 0
  
            #Display how the discounts are presented
            discount_pattern = re.compile(r"_[1-9][0-9]off_", flags = re.I)
            discount_mask = df_sample["campaign_type"].str.contains(discount_pattern, regex=True)  
            df_sample.loc[discount_mask, "discount"] = (df_sample[discount_mask])["campaign_type"].str.findall(r"(?<=_)\d{2}").str.get(0).astype(int)

            #Check whether promo/sale
            df_sample["promo"] = df_sample.campaign_type.str.contains("Promo", case=False).fillna(0).astype(int)
            df_sample["sale"] = df_sample.campaign_type.str.contains("Sale", case=False).fillna(0).astype(int)

            #Drop the campaign_type column 
            df_sample.drop(columns=["campaign_type"], inplace=True)

        if control_variable['offer_cat']:

            #Drop this column as well
            df_sample.drop(columns=["offer_category"], inplace=True)

#        if control_variable["aq"]:
#            col_drop_aq = ["aq_dt", "aq_mo", "aq_dow", "aq_period"]
#            df_sample.drop(columns=col_drop_aq, inplace=True)

        if control_variable["site"]:
            col_drop_site = ["wk1_vis", "wk4_vis", "mo3_vis", "mo6_vis", "wk1_pv", "wk4_pv", "mo3_pv", "mo6_pv", "wk1_secs", "wk4_secs", "mo3_secs", "mo6_secs"]
            df_sample.drop(columns=col_drop_site, inplace=True)

        if control_variable["last"]:
            col_drop_last = ["last_order", "last_click", "last_cart","last_visit","last_ded_open"]
            df_sample.drop(columns=col_drop_last, inplace=True)

        if control_variable["promo_ids"]:
            col_drop_promo_ids = ["offer_signature_id", "launch_id"] #, "campaign_id"]
            df_sample.drop(columns=col_drop_promo_ids, inplace=True)

        if control_variable["email_cnts"]:
            col_drop_email_cnts = ["open_cnt", "click_cnt"]
            df_sample.drop(columns=col_drop_email_cnts, inplace=True)

        if control_variable["email_times"]:
            col_drop_email_times = ["first_open_time", "last_open_time", "hrs_to_first_open", "hrs_to_last_open", "first_click_time", "last_click_time", "hrs_to_first_click", "hrs_to_last_click"]
            df_sample.drop(columns=col_drop_email_times, inplace=True)

        if control_variable["opt_out"]:
            col_drop_opt_out = ["prev_optouts", "first_opt_out_time", "opt_out_cnt", "last_opt_out_time", "opt_out_source",  "opt_out_reason", "hrs_to_first_opt_out", "hrs_to_last_opt_out"]
            df_sample.drop(columns=col_drop_opt_out, inplace=True)

        if control_variable["subject"]:
            df_sample.drop(columns=["subject"], inplace=True)
        
        #Add the reward column
        df_sample['reward'] = df_sample.apply(lambda row: rew(row), axis=1)
        #Add optimal action column
        df_sample['optimal_action'] = df_sample.apply(lambda row: optimal(row), axis=1)
     
        df_sample.drop(columns=["sends_since_last_open"], inplace=True)

        df_sample = df_sample[ordered_cols]
        df_sample.sort_values(by=ordered_rows, inplace=True)
    
        df_sample.to_csv(path_or_buf = filename+'_'+f'{ind:02}'+".csv", index=False, header=False)

/home/aleksey/Downloads/CSV_DATA/Jul_Dec_2019/sends_july_dec_2019_2000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
/home/aleksey/Downloads/CSV_DATA/Jan_Jun_2020/sends_jan_june_2020_2000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
/home/aleksey/Downloads/CSV_DATA/Jul_Sep_2020/sends_july_sept_2020_2000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [50]:
# Split each of the files with ~10**6 lines into files with emails sent each week
directories = ['/home/aleksey/Downloads/CSV_DATA/Jul_Dec_2019/', '/home/aleksey/Downloads/CSV_DATA/Jan_Jun_2020/', '/home/aleksey/Downloads/CSV_DATA/Jul_Sep_2020/']
for directory in directories:
    print ("Processing", directory)
    for filename in sorted(os.listdir(directory)):
        if '.csv' in filename:
            print (filename)
            df = pd.read_csv(directory+filename, header=None, names=ordered_cols)
            weeks = df['sent_week'].unique()
            for week in weeks:
                df_sample = df.loc[df['sent_week'] == week]
                df_sample.to_csv(directory+filename+"_"+f'{week:02}', index=False, header=False)

Processing /home/aleksey/Downloads/CSV_DATA/Jul_Dec_2019/
sends_july_dec_2019_2000_01.csv
sends_july_dec_2019_2000_02.csv
sends_july_dec_2019_2000_03.csv
sends_july_dec_2019_2000_04.csv
sends_july_dec_2019_2000_05.csv
sends_july_dec_2019_2000_06.csv
sends_july_dec_2019_2000_07.csv
sends_july_dec_2019_2000_08.csv
sends_july_dec_2019_2000_09.csv
sends_july_dec_2019_2000_10.csv
sends_july_dec_2019_2000_11.csv
sends_july_dec_2019_2000_12.csv
sends_july_dec_2019_2000_13.csv
sends_july_dec_2019_2000_14.csv
sends_july_dec_2019_2000_15.csv
sends_july_dec_2019_2000_16.csv
sends_july_dec_2019_2000_17.csv
sends_july_dec_2019_2000_18.csv
sends_july_dec_2019_2000_19.csv
sends_july_dec_2019_2000_20.csv
sends_july_dec_2019_2000_21.csv
sends_july_dec_2019_2000_22.csv
sends_july_dec_2019_2000_23.csv
sends_july_dec_2019_2000_24.csv
sends_july_dec_2019_2000_25.csv
sends_july_dec_2019_2000_26.csv
sends_july_dec_2019_2000_27.csv
sends_july_dec_2019_2000_28.csv
sends_july_dec_2019_2000_29.csv
sends_july_dec

In [59]:
# Use bash scripts to concatenate all files with emails for the same week
for directory in directories:
    os.chdir(directory)
    subprocess.call(directory+"format.sh")

In [60]:
# Save files with emails for each week, sorted by week --> day of week --> hour sent --> campaign ID
directories = ['/home/aleksey/Downloads/CSV_DATA/Jul_Dec_2019/', '/home/aleksey/Downloads/CSV_DATA/Jan_Jun_2020/', '/home/aleksey/Downloads/CSV_DATA/Jul_Sep_2020/']
for directory in directories:
    print ("Processing", directory)
    for filename in sorted(os.listdir(directory)):
        if '.out' in filename:
            if '2019' in filename:
                outfile='sends_2019_wk'+filename[-6:-4]+".csv"
            else:
                outfile='sends_2020_wk'+filename[-6:-4]+".csv"
            print (outfile)
            df = pd.read_csv(directory+filename, header=None, names=ordered_cols)
            df.sort_values(by=ordered_rows, inplace=True)
            df.to_csv(directory+outfile, index=False)

Processing /home/aleksey/Downloads/CSV_DATA/Jul_Dec_2019/
sends_2019_wk26.csv
sends_2019_wk27.csv
sends_2019_wk28.csv
sends_2019_wk29.csv
sends_2019_wk30.csv
sends_2019_wk31.csv
sends_2019_wk32.csv
sends_2019_wk33.csv
sends_2019_wk34.csv
sends_2019_wk35.csv
sends_2019_wk36.csv
sends_2019_wk37.csv
sends_2019_wk38.csv
sends_2019_wk39.csv
sends_2019_wk40.csv
sends_2019_wk41.csv
sends_2019_wk42.csv
sends_2019_wk43.csv
sends_2019_wk44.csv
sends_2019_wk45.csv
sends_2019_wk46.csv
sends_2019_wk47.csv
sends_2019_wk48.csv
sends_2019_wk49.csv
sends_2019_wk50.csv
sends_2019_wk51.csv
sends_2019_wk52.csv
Processing /home/aleksey/Downloads/CSV_DATA/Jan_Jun_2020/
sends_2020_wk00.csv
sends_2020_wk01.csv
sends_2020_wk02.csv
sends_2020_wk03.csv
sends_2020_wk04.csv
sends_2020_wk05.csv
sends_2020_wk06.csv
sends_2020_wk07.csv
sends_2020_wk08.csv
sends_2020_wk09.csv
sends_2020_wk10.csv
sends_2020_wk11.csv
sends_2020_wk12.csv
sends_2020_wk13.csv
sends_2020_wk14.csv
sends_2020_wk15.csv
sends_2020_wk16.csv
send